In [9]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [13]:
overall_df = pd.DataFrame(columns=["note1", "note2", "note3", "note4", "note5", "ABDOMINAL",
"ADVANCED-CAD",
"ALCOHOL-ABUSE",
"ASP-FOR-MI",
"CREATININE",
"DIETSUPP-2MOS",
"DRUG-ABUSE",
"ENGLISH",
"HBA1C",
"KETO-1YR",
"MAJOR-DIABETES",
"MAKES-DECISIONS",
"MI-6MOS"])

current_directory = os.getcwd()
folder_name = 'train'
directory = os.path.join(current_directory, folder_name)
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        patient_num = os.path.splitext(filename)[0]
        row_to_add = {}
        # Load the XML file
        tree = ET.parse(os.path.join(directory, filename))
        root = tree.getroot()
        # Access elements and attributes
        for child in root:
            if child.tag == "TEXT":
                notes = child.text.split("****************************************************************************************************")
                notes = [note.strip() for note in notes if note.strip()]
                i = 1
                for note in notes:
                    row_to_add[f"note{i}"] = note
                    i += 1
                for j in range(i, 6):
                    row_to_add[f"note{j}"] = ""
            if child.tag == "TAGS":
                for subchild in child:
                    row_to_add[subchild.tag] = 1 if subchild.attrib.get('met') == 'met' else 0
        overall_df.loc[patient_num] = row_to_add

overall_df.head(5)
        




(202, 18)

In [5]:
from transformers import BertTokenizer, TFBertForTokenClassification
import tensorflow as tf

# Load BioBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")

# Load BioBERT model for named entity recognition (NER)
model = TFBertForTokenClassification.from_pretrained("monologg/biobert_v1.1_pubmed", num_labels=2)

# Clinical note data
clinical_note = "The patient underwent abdominal surgery for appendicitis last week. Recovery has been uneventful."

# Tokenize clinical note
tokenized_input = tokenizer(clinical_note, return_tensors="tf")

# Perform inference
outputs = model(tokenized_input)

# Retrieve predicted labels
predicted_labels = tf.argmax(outputs.logits, axis=-1)[0].numpy()

# Define the mapping of label indices to entity types
label_map = {0: "O", 1: "B-SURG", 2: "I-SURG"}  # Assuming BIO tagging scheme for surgery entities

# Extract surgery entities
surgery_entities = []
current_entity = ""
for token, label_idx in zip(tokenizer.tokenize(clinical_note), predicted_labels):
    label = label_map[label_idx]
    if label == "B-SURG":
        current_entity = token
    elif label == "I-SURG":
        current_entity += " " + token
    elif label == "O" and current_entity:
        surgery_entities.append(current_entity)
        current_entity = ""

# Check if abdominal surgery is mentioned
abdominal_surgery_mentioned = any("abdominal surgery" in entity.lower() for entity in surgery_entities)

# Print the result
if abdominal_surgery_mentioned:
    print("The clinical note mentions that the patient underwent abdominal surgery.")
else:
    print("No mention of abdominal surgery found in the clinical note.")

OSError: https://tfhub.dev/google/BioBERT_Large_TuneD_PubMed/2 does not appear to be a valid module.